<a href="https://colab.research.google.com/github/shikhaup/flask-app/blob/main/meter_flask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask-ngrok
!pip install flask==0.12.2  # Newer versions of flask don't work in Colab
                            # See https://github.com/plotly/dash/issues/257

     |████████████████████████████████| 92kB 7.6MB/s 
  Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


In [ ]:
%%writefile /content/templates/home.html
<!DOCTYPE html>
<html>
<head>
 <title>Meter Reading App</title>
</head>
<body>
 <h2>Upload your meter image</h2>
 <form method ='post' enctype=multipart/form-data>
  <input type="file" name="file">
  <input type="submit" value="upload">
</body>
</html>

Overwriting /content/templates/home.html


In [ ]:
%%writefile /content/templates/result.html
<!DOCTYPE html>
<html>
<head>
	<title>Meter Reading App</title>
</head>
<body>
  {% if filename %}
	<div>
		<img src="{{ url_for('display_image', filename=filename) }}">
	</div>
{% endif %}
	<h2>Prediction</h2>
	<p>Reading: {{ reading }}</p>
</body>
</html>

Overwriting /content/templates/result.html


In [ ]:
%%writefile app.py
import torch
from flask_ngrok import run_with_ngrok
from flask import Flask
from PIL import Image
from flask import Flask, render_template, request, redirect, url_for,flash
import string
import os
from CRNN.utils.utils import strLabelConverter, decode_prediction
from inference import dataextraction, ImageProcessing

alphabet = string.digits
label_converter = strLabelConverter(alphabet)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

UPLOAD_FOLDER = '/content/static/uploads/'


app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.secret_key = "shikhaaa"

@app.route('/', methods=['GET', 'POST'])
def hello_world():
  if request.method == 'GET':
    return render_template('home.html', value='hi')
  if request.method == 'POST':
    print(request.files)
    file = request.files['file']
    filename = file.filename
    img = file.read()
    
    file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    
    flash('Image successfully uploaded and displayed')

    img = ImageProcessing(img)
    pred_text = dataextraction(img, device, label_converter)
    print(pred_text)
    return render_template('result.html', reading=pred_text, filename=filename)

@app.route('/display/<filename>')
def display_image(filename):
	#print('display_image filename: ' + filename)
	return redirect(url_for('static', filename='uploads/' + filename), code=301)

if __name__ == '__main__':
  app.run()

Overwriting app.py


In [ ]:
!python app.py

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Running on http://c8c7a3a38828.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
127.0.0.1 - - [30/Dec/2020 18:14:57] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2020 18:14:58] "GET /favicon.ico HTTP/1.1" 404 -
ImmutableMultiDict([('file', <FileStorage: '03048.png' ('image/png')>)])
03
127.0.0.1 - - [30/Dec/2020 18:15:20] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [30/Dec/2020 18:15:20] "GET /display/03048.png HTTP/1.1" 301 -
127.0.0.1 - - [30/Dec/2020 18:15:21] "GET /static/uploads/03048.png HTTP/1.1" 200 -


In [ ]:
%%writefile /content/CRNN/utils/utils.py
import collections
import torch
import torch.nn as nn
from numpy import mean




class strLabelConverter(object):
    """Convert between str and label.
        Insert `blank` to the alphabet for CTC.
    Args:
        alphabet (str): set of the possible characters.
        ignore_case (bool, default=True): whether or not to ignore all of the case.
    """

    def __init__(self, alphabet: str, ignore_case: bool = True):
        self._ignore_case = ignore_case
        if self._ignore_case:
            alphabet = alphabet.lower()
        self.alphabet = alphabet + '-'  # for `-1` index

        self.char2idx = {}
        for i, char in enumerate(alphabet):
            # NOTE: 0 is reserved for 'blank' required by wrap_ctc
            self.char2idx[char] = i + 1
        self.idx2char = {idx: char for char, idx in self.char2idx.items()}

    def encode(self, text):
        """Support batch or single str.
        Args:
            text (str or list of str): texts to convert.
        Returns:
            torch.IntTensor [length_0 + length_1 + ... length_{n - 1}]: encoded texts.
            torch.IntTensor [n]: length of each text.
        """
        # print(text)
        if isinstance(text, str):
            text = [
                self.char2idx[char.lower() if self._ignore_case else char]
                for char in text
            ]
            length = [len(text)]
        elif isinstance(text, collections.Iterable):
            length = [len(s) for s in text]
            text = ''.join(text)
            text, _ = self.encode(text)
        return (torch.IntTensor(text), torch.IntTensor(length))

    def decode(self, t, length, raw=False):
        """Decode encoded texts back into strs.
        Args:
            torch.IntTensor [length_0 + length_1 + ... length_{n - 1}]: encoded texts.
            torch.IntTensor [n]: length of each text.
        Raises:
            AssertionError: when the texts and its length does not match.
        Returns:
            text (str or list of str): texts to convert.
        """
        if length.numel() == 1:
            length = length[0]
            assert t.numel() == length, "text with length: {} does not match declared length: {}".format(t.numel(), length)
            if raw:
                return ''.join([self.alphabet[i - 1] for i in t])
            else:
                char_list = []
                for i in range(length):
                    if t[i] != 0 and (not (i > 0 and t[i - 1] == t[i])):
                        char_list.append(self.alphabet[t[i] - 1])
                return ''.join(char_list)
        else:
            # batch mode
            assert t.numel() == length.sum(), "texts with length: {} does not match declared length: {}".format(t.numel(), length.sum())
            texts = []
            index = 0
            for i in range(length.numel()):
                l = length[i]
                texts.append(
                    self.decode(
                        t[index:index + l], torch.IntTensor([l]), raw=raw))
                index += l
        return texts


def decode_prediction(logits: torch.Tensor, 
                      label_converter: strLabelConverter) -> str:
    tokens = logits.softmax(2).argmax(2)
    tokens = tokens.squeeze(1).numpy()
    
    # convert tor stings tokens
    tokens = ''.join([label_converter.idx2char[token] 
                      if token != 0  else '-' 
                      for token in tokens])
    tokens = tokens.split('-')
    
    # remove duplicates
    text = [char 
            for batch_token in tokens 
            for idx, char in enumerate(batch_token)
            if char != batch_token[idx-1] or len(batch_token) == 1]
    text = ''.join(text)
    return text

Overwriting /content/CRNN/utils/utils.py


In [ ]:
!python /content/CRNN/utils/utils.py

In [ ]:
%%writefile inference.py
from __future__ import division
import io

import torch
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import cv2
import string
import torchvision.transforms as transforms
from CRNN.utils.utils import strLabelConverter, decode_prediction

def get_model():
  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
  path = Path('/content/save.pt')
  crnn = torch.load(path)
  crnn.eval()
  return crnn

# This is the alternate function for get_tensor fucntion.... 
def ImageProcessing(image_bytes):
  data_transform2 = transforms.Compose([transforms.ToTensor()])
  img = Image.open(io.BytesIO(image_bytes))
  # now we will return the tensor version of that image with resize .. as our input should be 300*75 resolution,
  # if we want PIL format then we have to return without transformation and again have to do the resize in dataextraction 
  return data_transform2(img)
  
def dataextraction(img, device, label_converter):
    # data_transform = transforms.Compose([transforms.ToTensor()])
    # img = Image.open(io.BytesIO(img))
    # img = data_transform(img)  #We don't have to do that now, as we already converted our image to tensor on Imageprocessing function.
    crnn = get_model()
    with torch.no_grad():
        # crnn.eval()
        img = img.unsqueeze(0)
        logits = crnn(img.to(device))

    pred_text = decode_prediction(logits.cpu(), label_converter)
    return pred_text

#crnn = get_model()
#print(crnn.eval())

Overwriting inference.py


In [ ]:
!python inference.py

In [ ]:

def get_model():
	checkpoint_path = 'checkpoint_final.pth'
	model = models. densenet161(pretrained=True)
	model.classifier = nn.Linear(2208, 102)
	model.load_state_dict(torch.load(
		checkpoint_path, map_location='cpu'), strict=False)
	model.eval()
	return model

In [ ]:
# This is the alternate function for get_tensor fucntion.... 
def ImageProcessing(image_bytes):
  data_transform2 = transforms.Compose([transforms.Resize((300,75)),
                                       transforms.ToTensor()])
  img = Image.open(io.BytesIO(image_bytes))
  # now we will return the tensor version of that image with resize .. as our input should be 300*75 resolution,
  # if we want PIL format then we have to return without transformation and again have to do the resize in dataextraction 
  return data_transform2(img)